<a href="https://colab.research.google.com/github/KarineAyrs/science_work/blob/main/training/PyTorchCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [12]:
class CNN(nn.Module):
  def __init__(self, in_channel=1, num_classes=10):
    super(CNN, self).__init__()
    self.conv1=nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1)) # same convolution
    self.pool=nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
    self.conv2=nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1)) # same convolution
    self.fc1=nn.Linear(16*7*7, num_classes)
  
  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0],-1)
    x = self.fc1(x)

    return x



In [13]:
model = CNN()
x = torch.randn(64,1,28,28)
print(model(x).shape)

torch.Size([64, 10])


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
in_channel=1
num_classes=10
learning_rate=0.001
batch_size=64
num_epochs =5

In [16]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader=DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader=DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [17]:
model=CNN().to(device)

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
for epoch in range(num_epochs):
  for batch, (data, targets) in enumerate(train_loader):
    # Get data to cuda if possible
    data=data.to(device=device)
    targets=targets.to(device=device)
    
    # forward
    scores = model(data)
    loss = criterion(scores, targets)

    # backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent or adam step
    optimizer.step()



In [20]:
# Check accuracy on training & test to see how good our model

def check_accuracy(loader, model):
  if loader.dataset.train:
    print('Checking accuracy on training data')
  else:
    print('Checking accuracy o test data')
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x=x.to(device=device)
      y=y.to(device=device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct+=(predictions==y).sum()
      num_samples+=predictions.size(0)
    
    print(f'Got {num_correct}/{num_samples} with accuracy {(float(num_correct)/float(num_samples))*100}')
  
  model.train()
 
  

In [21]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 59091/60000 with accuracy 98.485
Checking accuracy o test data
Got 9840/10000 with accuracy 98.4
